In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge,Lasso,RidgeCV,LassoCV,ElasticNetCV,ElasticNet,LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from pandas_profiling import ProfileReport
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pickle

In [ ]:
df = pd.read_csv('ai4i2020.csv')
df.head()

In [ ]:
pf = ProfileReport(df)
pf.to_widgets()
pf.to_file("templates/report.html")

In [ ]:
df=df.iloc[:,3:]
df.head()

In [ ]:
df.describe()

In [ ]:
x = df.drop(columns=['Air temperature [K]'])
x.head()

In [ ]:
y=df['Air temperature [K]']
y.head()

In [ ]:
scaler = StandardScaler()

In [ ]:
arr = scaler.fit_transform(x)
arr

In [ ]:
df1 = pd.DataFrame(arr)
df1.head()

In [ ]:
vif_df = pd.DataFrame()
vif_df['vif'] = [variance_inflation_factor(arr,i)for i in range(arr.shape[1])]
vif_df['feature'] = x.columns
vif_df

In [ ]:
# Here VIF value of Machine failure column is greater than 10 so we drop column

In [ ]:
x = x.drop(columns=['Machine failure'])
x.head()

In [ ]:
arr1 = scaler.fit_transform(x)
arr1

In [ ]:
df1 = pd.DataFrame(arr1)
df1.head()

In [ ]:
vif_df1 = pd.DataFrame()
vif_df1['vif'] = [variance_inflation_factor(arr1,i)for i in range(arr1.shape[1])]
vif_df1['feature'] = x.columns
vif_df1

In [ ]:
X_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.25,random_state = 20)

In [ ]:
X_train

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
lr.score(x_test,y_test)

In [ ]:
def adj_r2(x,y):
    r2 = lr.score(x,y)
    n = x.shape[0]
    p = x.shape[1]
    adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    return adjusted_r2

In [ ]:
adj_r2(x_test,y_test)

In [ ]:
lr.coef_

In [ ]:
lr.intercept_

In [ ]:
lassocv = LassoCV(alphas=None,cv = 10,max_iter=20000000,normalize=True)
lassocv.fit(X_train,y_train)

In [ ]:
lassocv.alpha_

In [ ]:
lass = Lasso(alpha=lassocv.alpha_)
lass.fit(X_train,y_train)

In [ ]:
lass.score(x_test,y_test)

In [ ]:
alpha = np.random.uniform(0,10,50)

In [ ]:
ridgecv = RidgeCV(alphas=alpha,cv = 10,normalize=True)
ridgecv.fit(X_train,y_train)

In [ ]:
ridgecv.alpha_

In [ ]:
ridge_lr = Ridge(alpha=ridgecv.alpha_)
ridge_lr.fit(X_train,y_train)

In [ ]:
ridge_lr.score(x_test,y_test)

In [ ]:
elastic = ElasticNetCV(alphas=None,cv = 10)
elastic.fit(X_train,y_train)

In [ ]:
elastic.alpha_

In [ ]:
elastic.l1_ratio_

In [ ]:
elastic_lr = ElasticNet(alpha=elastic.alpha_,l1_ratio=elastic.l1_ratio_)

In [ ]:
elastic_lr.fit(X_train,y_train)

In [ ]:
elastic_lr.score(x_test,y_test)

In [ ]:
result= {"Linear Regression Score: ":lr.score(x_test,y_test),
         "Lasso Regression Score: ":lass.score(x_test,y_test),
         "Ridge Regression Score: ":ridge_lr.score(x_test,y_test),
         "ElasticNet Regression Score: ":elastic_lr.score(x_test,y_test)}

In [ ]:
print(result)

In [ ]:
lr.predict([[308.7,1408,46.3,3,0,0,0,0,0]])

In [ ]:
elastic_lr.predict([[308.7,1408,46.3,3,0,0,0,0,0]])

In [ ]:
lass.predict([[308.7,1408,46.3,3,0,0,0,0,0]])

In [ ]:
pickle.dump(lass,open('challenge.pickle','wb'))

In [ ]:
model =  pickle.load(open('challenge.pickle','rb'))

In [ ]:
model.predict([[308.7,1408,46.3,3,0,0,0,0,0]])

In [ ]:
!pip freeze > requirements.txt